In [1]:
import requests

In [65]:
url = 'https://slack.com/api'
# load in my slack API token
with open('token.txt', 'r') as t:
    token = t.read()

Need to get the channel ID for resources, uses channels.list.  Go ahead and make a dict of all channel names with id's as values.

In [38]:
payload = {
    'token' : token
}

r = requests.get(url='{}/channels.list'.format(url), params=payload)

import json

response = json.loads(r.text)
channels = {item['name'] : item['id'] for item in response['channels']}

In [39]:
resources_id = channels['resources']

Now I can go through and pull the messages from that channel.  I will have to continually make requests to pull more messages until I have them all.

In [52]:
response['messages'][-1]

{'attachments': [{'fallback': 'Simple tool/library to visualize huge python dict',
   'from_url': 'http://stackoverflow.com/a/15024168/6661502',
   'id': 1,
   'service_icon': 'http://cdn.sstatic.net/Sites/stackoverflow/img/favicon.ico?v=4f32ecc8f43d',
   'service_name': 'stackoverflow.com',
   'text': "I have a huge dict structure like this one:my_data = { 'key1': { '_': 'value1': 'aaa' }, 'key2': { '_': 'value2': 'bbb', 'key2.1': { '_': 'ccc', ...",
   'thumb_height': 316,
   'thumb_url': 'http://cdn.sstatic.net/Sites/stackoverflow/img/apple-touch-icon@2.png?v=73d79a89bded&a',
   'thumb_width': 316,
   'title': 'Simple tool/library to visualize huge python dict',
   'title_link': 'http://stackoverflow.com/a/15024168/6661502'}],
 'text': 'I was looking for a way to display and interact with large dicts today and got this answer to work: <http://stackoverflow.com/a/15024168/6661502>',
 'ts': '1478923302.000061',
 'type': 'message',
 'user': 'U1RJ84JMD'}

In [57]:
# empty list to contain our dicts
message_dicts = []

# kick off a first request
payload = {
    'token' : token,
    'channel' : resources_id
}

r = requests.get(url='{}/channels.history'.format(url), params=payload)
response = json.loads(r.text)

message_dicts += response['messages']

# Are there more messages to pull?
if response['has_more']:
    done = False
else:
    done = True
    
while not done:
    if response['has_more'] == False:
        done = True
    # get ts value for last message id in list to pass as latest param in next call
    latest = response['messages'][-1]['ts']
    
    payload = payload = {
        'token' : token,
        'channel' : resources_id,
        'latest' : latest
    }
    
    r = requests.get(url='{}/channels.history'.format(url), params=payload)
    response = json.loads(r.text)
    
    # add message dicts to list
    message_dicts += response['messages']

Writing the JSON out to file to read in later and parse more stuff

In [95]:
dump = json.dumps(message_dicts)

with open('message_dicts.json', 'w') as m:
    m.write(dump)

Here's the code to read it back in.  Make it a code cell to activate it.

Now to try pulling the URL's from each message into a list for further parsing...where we can

In [74]:
attachments = [item.setdefault('attachments', None) for item in message_dicts]

In [80]:
urls = [[item[0]['title'], item[0]['from_url']] for item in attachments if item != None]

In [93]:
to_write = ['{},{}\n'.format(url[0],url[1]) for url in urls]

In [94]:
with open('urls.txt', 'w') as u:
    u.writelines(to_write)

In [88]:
urls

[['csurfer/pyheat', 'https://github.com/csurfer/pyheat'],
 ['Improving your code with container methods - Python Programming',
  'https://python-programming.courses/code-mastery/improving-code-container-methods/'],
 ["Hacker's guide to Neural Networks",
  'http://karpathy.github.io/neuralnets/'],
 ['jakevdp/PythonDataScienceHandbook',
  'https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/README.md'],
 ['pandas-dev/pandas',
  'https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf'],
 ['airbnb/superset', 'https://github.com/airbnb/superset'],
 ['lk-geimfari/awesomeo',
  'https://github.com/lk-geimfari/awesomeo/blob/master/languages/PYTHON.md'],
 ['obachem/kmc2',
  'https://github.com/obachem/kmc2?__hstc=36392319.c7989f645cc3265cd4938b505d0c5512.1482440390003.1482440390003.1482440390003.1&__hssc=36392319.1.1482440390003&__hsfp=259699018'],
 ['An Intuitive Explanation of Convolutional Neural Networks | Open Data Science',
  'https://www.openda